<a href="https://colab.research.google.com/github/kleczekr/tolkenizer/blob/master/violence9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline 

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # needed for waffle Charts
# import seaborn as sns
import numpy as np
import folium

In [2]:
# This cell is meant to accommodate the Google Colab way of dealing with reading 
# files from Google Drive; feel free to ignore it if you are running the notebook
# on your local machine
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_excel('/content/drive/My Drive/ethnic_violence/globalterrorismdb_0919dist.xlsx')

In [4]:
# save the data pertaining the Eelam War I in Sri Lanka as a separate dataframe
condition = ((df.country_txt == 'Sri Lanka') & (df.iyear >= 1983) & (df.iyear <= 1987))
df_sl_first = df[condition]

In [5]:
df_sl_first.shape

(455, 135)

In [6]:
pd.set_option('display.max_rows', 150)

In [7]:
df_sl_first.isnull().sum()

eventid                 0
iyear                   0
imonth                  0
iday                    0
approxdate            454
extended                0
resolution            451
country                 0
country_txt             0
region                  0
region_txt              0
provstate               0
city                    0
latitude               35
longitude              35
specificity             0
vicinity                0
location              447
summary               447
crit1                   0
crit2                   0
crit3                   0
doubtterr               0
alternative           346
alternative_txt       346
multiple                0
success                 0
suicide                 0
attacktype1             0
attacktype1_txt         0
attacktype2           454
attacktype2_txt       454
attacktype3           455
attacktype3_txt       455
targtype1               0
targtype1_txt           0
targsubtype1           18
targsubtype1_txt       18
corp1       

What is of particular importance for us in the summary of missing data above (since we are planning to plot the data on a map) is the 35 missing latitude and longitude values. If the events with missing geographical information are comparatively insignificant, we could simply remove them from the dataframe---but are they? It is easy to check---let's take a closer look at these 35 events:

In [8]:
pd.set_option('display.max_colwidth', 150)

In [9]:
df_sl_first[df_sl_first.latitude.isnull()]

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,...,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
18199,198303000004,1983,3,0,NaN,0,NaT,186,Sri Lanka,6,South Asia,Unknown,Unknown,NaN,NaN,5.0,0,NaN,NaN,1,1,0,1,1.0,Insurgency/Guerilla Action,0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,4,Military,35.0,Military Transportation/Vehicle (excluding convoys),NaN,Army Truck,...,NaN,NaN,Ambushed; Firearm,0.0,0.0,NaN,5.0,0.0,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,0,-9,NaN
21881,198404300004,1984,4,30,NaN,0,NaT,186,Sri Lanka,6,South Asia,Unknown,Unknown,NaN,NaN,5.0,0,NaN,NaN,1,1,1,-9,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,3,Police,25.0,Police Security Forces/Officers,Kelaiya police station,Police constable Pavilingam Pillai,...,NaN,NaN,Firearm,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,0,-9,NaN
22257,198406100009,1984,6,10,NaN,0,NaT,186,Sri Lanka,6,South Asia,Unknown,Unknown,NaN,NaN,5.0,0,NaN,NaN,1,1,1,-9,NaN,NaN,0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,3,Police,26.0,Prison/Jail,NaN,Batticoloa Prison,...,NaN,NaN,Automatic firearms; Revolvers,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,7.0,Unknown,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,0,1,NaN
22887,198408110003,1984,8,11,NaN,0,NaT,186,Sri Lanka,6,South Asia,Unknown,Unknown,NaN,NaN,5.0,0,NaN,NaN,1,1,0,1,1.0,Insurgency/Guerilla Action,0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,4,Military,29.0,Military Unit/Patrol/Convoy,NaN,Army Patrol,...,NaN,NaN,Explosive,0.0,NaN,NaN,2.0,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,0,-9,NaN
22889,198408110005,1984,8,11,NaN,0,NaT,186,Sri Lanka,6,South Asia,Unknown,Tirunachchi,NaN,NaN,5.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,9,Food or Water Supply,51.0,Food Supply,NaN,Lorry of coconuts and food items,...,NaN,NaN,Land mine,2.0,NaN,NaN,0.0,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,0,-9,NaN
23784,198411180002,1984,11,18,NaN,0,NaT,186,Sri Lanka,6,South Asia,Unknown,unknown,NaN,NaN,5.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,3,Police,23.0,Police Patrol (including vehicles and convoys),NaN,police commando vehicle,...,NaN,NaN,Explosive,0.0,NaN,NaN,5.0,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,0,1,NaN
23892,198412030004,1984,12,3,NaN,0,NaT,186,Sri Lanka,6,South Asia,Unknown,Unknown,NaN,NaN,5.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,5,Hostage Taking (Barricade Incident),NaN,NaN,NaN,NaN,19,Transportation,100.0,Train/Train Tracks/Trolley,NaN,Train,...,NaN,NaN,Attacked,0.0,NaN,NaN,0.0,NaN,NaN,1,NaN,NaN,NaN,NaN,1.0,9.0,0.0,NaN,NaN,NaN,NaN,1.0,400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
23982,198412190004,1984,12,19,NaN,0,NaT,186,Sri Lanka,6,South Asia,Unknown,unknown,NaN,NaN,5.0,1,NaN,NaN,1,1,0,1,1.0,Insurgency/Guerilla Action,0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,4,Military,35.0,Military Transportation/Vehicle (excluding convoys),NaN,2 trucks taking 25 soldiers to anothe

In [10]:
# df_sl_first.tail(30)

I think the 35 incidents lacking precise geographical location are a valuable part of our data. In order to include them in a visualization in the form of a map of incidents we could theoretically mark the latitude and longitude at the center of the country---Sri Lanka. The question we need to answer before we do that is: would this be a valuable addition to our visualization? I think that an agglomeration of incidents in the very center of the country would give the readers a wrong impression that the center of the map is somehow a place of heightened military activity---to some extent, this would invalidate what we are trying to accomplish with the visualization. While it is always a difficult decision, I think we need to dispose of the 35 incidents missing precise geographical location in order to accurately depict the events.

In [11]:
df_sl_first = df_sl_first[df_sl_first.latitude.notnull()]

In [12]:
df_sl_first.shape

(420, 135)

In [13]:
# Sri Lanka's latitude and longitude values
latitude = 7.8731
longitude = 80.7718

In [14]:
# create map and display it
sl_map = folium.Map(location=[latitude, longitude], zoom_start=8)

# display the map of San Francisco
sl_map

In [15]:
from folium import plugins

# let's start again with a clean copy of the map of Sri Lanka
sa = folium.Map(location = [latitude, longitude], zoom_start = 7)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sl_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, attacktype, nkill, targtype, targsubtype, perpetrator in zip(df_sl_first.latitude, 
                                         df_sl_first.longitude, 
                                         df_sl_first.attacktype1_txt, 
                                         df_sl_first.nkill, 
                                         df_sl_first.targtype1_txt,
                                         df_sl_first.targsubtype1_txt,
                                         df_sl_first.gname):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup='Attack type: {}<br> Number of victims: {}<br> Target: {}<br> \
        Target subtype: {}<br> Perpetrator: {}'.format(attacktype,
                                                     nkill,
                                                     targtype,
                                                     targsubtype,
                                                     perpetrator),
    ).add_to(incidents)

# display map
sl_map